In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque
from sklearn.neighbors import NearestNeighbors

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
df1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2014-2015.csv')
df1.Date = pd.to_datetime(df1.Date)

df2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2015-2016.csv')
df2.Date = pd.to_datetime(df2.Date)

dfs1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2014-2015.csv')
dfs1.Date = pd.to_datetime(dfs1.Date)

dfs2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2015-2016.csv')
dfs2.Date = pd.to_datetime(dfs2.Date)

df = df1.append(df2)
df.reset_index(drop=True, inplace=True)

df_small = dfs1.append(dfs2)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2014-07-30 07:00:00   1.69402   1.69402  1.69323    1.69332   1.69417   
1 2014-07-30 07:15:00   1.69332   1.69356  1.69278    1.69323   1.69348   
2 2014-07-30 07:30:00   1.69319   1.69351  1.69304    1.69343   1.69334   
3 2014-07-30 07:45:00   1.69343   1.69364  1.69314    1.69338   1.69359   
4 2014-07-30 08:00:00   1.69339   1.69361  1.69334    1.69348   1.69353   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.69417  1.69338    1.69347   1.69410   1.69410  1.69330    1.69340  
1   1.69372  1.69296    1.69337   1.69340   1.69364  1.69287    1.69330  
2   1.69367  1.69320    1.69357   1.69326   1.69359  1.69312    1.69350  
3   1.69379  1.69329    1.69354   1.69351   1.69372  1.69322    1.69346  
4   1.69376  1.69348    1.69363   1.69346   1.69368  1.69342    1.69356  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2014-07-30 07:00:00   1.69402   1.69402  1.69359    1.69360   1.69417   
1 2014-07-30 07:05:00   1.69360   1.69373  1.69344    1.69359   1.69375   
2 2014-07-30 07:10:00   1.69357   1.69382  1.69323    1.69332   1.69372   
3 2014-07-30 07:15:00   1.69332   1.69336  1.69278    1.69310   1.69348   
4 2014-07-30 07:20:00   1.69309   1.69356  1.69306    1.69345   1.69323   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.69417  1.69374    1.69374   1.69410   1.69410  1.69366    1.69367  
1   1.69388  1.69359    1.69372   1.69368   1.69380  1.69352    1.69366  
2   1.69398  1.69338    1.69347   1.69364   1.69390  1.69330    1.69340  
3   1.69352  1.69296    1.69323   1.69340   1.69344  1.69287    1.69316  
4   1.69372  1.69320    1.69361   1.69316   1.69364  1.69313    1.69353  
---------------------------------------------------------------------------
                      Date  B

In [6]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(row):
  macdhist, ema50, mid_low, mid_high, rsi = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High', 'rsi']]

  if float(macdhist) > 0 and float(mid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(mid_high) < float(ema50):
    return 2

  else:
    return 0

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())

KeyboardInterrupt: 

In [68]:
df_small['ema200'] = talib.EMA(df_small['Mid_Close'], timeperiod=200)
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)
df_small['rsi'] = talib.RSI(df_small['Mid_Close'], timeperiod=14)
df_small.dropna(inplace=True)
df_small.reset_index(drop=True, inplace=True)

In [69]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [70]:
class AatTuple:
    def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
        self.time = time
        self.pips_risked = pips_risked
        self.pips_gained = pips_gained
        self.total_profit = total_profit
        self.adx_assumption = adx_assumption
        self.ema_assumption = ema_assumption
        self.rsi_assumption = rsi_assumption

In [71]:
# Use trade type in KNN?
# Normalize the data in KNN?

In [72]:
training_data = []

In [73]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_training_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1, lam=0.5):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        current_training_points = []
        step = 0

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

          z_adx_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].adx_assumption * lam
          z_ema_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].adx_assumption * lam
          z_rsi_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].adx_assumption * lam

          adx_holds = int(curr_adx > adx_cutoff)
          ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
          rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

          adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
          ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
          rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

          current_training_points.append(AatTuple(step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption))

          step += 1
          j += 1
        
        training_data.extend(current_training_points)

    i = 2

    while i < len(df):          
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        trending = adx >= adx_cutoff

        process_trade = False

        if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [74]:
# ----------------------------------------------------------------------------------------------------
# Run training simulation
# ----------------------------------------------------------------------------------------------------
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_training_simulation(2, 10, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))

-2300.500000000016
Num buys: 72
Num sells: 56
Num trades: 128
Num wins: 40
Num losses: 88
Win streak: 4
Loss streak: 13


In [75]:
len(training_data)

22164

In [76]:
# Repeate, but with test data
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2016-2017.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2016-2017.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [77]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())

0    12739
1     5940
2     5855
Name: beep_boop, dtype: int64
0.0    18061
1.0     3345
2.0     3128
Name: fractal, dtype: int64


In [78]:
df_small['ema200'] = talib.EMA(df_small['Mid_Close'], timeperiod=200)
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)
df_small['rsi'] = talib.RSI(df_small['Mid_Close'], timeperiod=14)
df_small.dropna(inplace=True)
df_small.reset_index(drop=True, inplace=True)

In [79]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_regular_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1

    i = 2

    while i < len(df):          
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        trending = adx >= adx_cutoff

        process_trade = False

        if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [80]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_regular_simulation(2, 10, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))

-667.5000000000002
Num buys: 42
Num sells: 47
Num trades: 89
Num wins: 31
Num losses: 57
Win streak: 3
Loss streak: 6
Average pips risked: 0.00488629213483146


In [81]:
# class AatTuple:
#     def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
#         self.time = time
#         self.pips_risked = pips_risked
#         self.pips_gained = pips_gained
#         self.total_profit = total_profit
#         self.adx_assumption = adx_assumption
#         self.ema_assumption = ema_assumption
#         self.rsi_assumption = rsi_assumption

x_train = []
for tup in training_data:
    curr_x = [tup.time, tup.pips_risked, tup.pips_gained, tup.total_profit, tup.adx_assumption, tup.ema_assumption, tup.rsi_assumption]
    x_train.append(curr_x)

x_train = np.array(x_train)


def distance_func(x, y):
    time_dist = 1000 * abs(x[0] - y[0])
    curr_pips_dist = 2 * abs(x[2] - y[2])
    pips_risked_dist = 3 * abs(x[1] - y[1])
    adx_dist = abs(x[4] - y[4])
    ema_dist = abs(x[5] - y[5])
    rsi_dist = abs(x[6] - y[6])

    return sum([time_dist, curr_pips_dist, pips_risked_dist, adx_dist, ema_dist, rsi_dist])

knn = NearestNeighbors(n_neighbors=15, metric=distance_func)
knn.fit(x_train)

NearestNeighbors(metric=<function distance_func at 0x124965440>, n_neighbors=15)

In [84]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_test_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1, lam=0.5):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    aat_gains = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        current_tuples = []
        step = 0

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
            trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if (step + 1) % 3 == 0:
            pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

            z_adx_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].adx_assumption * lam
            z_ema_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].adx_assumption * lam
            z_rsi_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].adx_assumption * lam

            adx_holds = int(curr_adx > adx_cutoff)
            ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
            rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

            adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
            ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
            rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

            current_tuple = np.array([step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption])
            distances, neighbor_indices = knn.kneighbors(current_tuple.reshape(1, -1), 15)
            estimated_gain = 0
            distance_sum = 0

            for p in range(len(neighbor_indices[0])):
              l = neighbor_indices[0][p]
              l_dist = distances[0][p]
              estimated_gain += (x_train[l][3] / l_dist)
              distance_sum += 1 / l_dist

            estimated_gain /= distance_sum

            day_fees_now = calculate_day_fees(trade['start_date'], curr_date)

            if estimated_gain < pips_gained_so_far + day_fees_now:
              trade_amount = pips_gained_so_far * value_per_pip
              reward += trade_amount
              day_fees += day_fees_now

              aat_gains.append(trade_amount)

              n_wins += 1 if trade_amount >= 0 else 0
              n_losses += 1 if trade_amount < 0 else 0
              curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
              curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

              if curr_win_streak > win_streak:
                win_streak = curr_win_streak

              if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

              trade['end_date'] = get_end_date(j)

              break

          step += 1
          j += 1

    i = 2

    ten_percent = int(len(df) / 10)

    while i < len(df):  
        if i % ten_percent == 0:
          print(i)        
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        trending = adx >= adx_cutoff

        process_trade = False

        if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, aat_gains

In [85]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, aat_gains = run_test_simulation(2, 10, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
print('Average aat gains: ' + str(sum(aat_gains) / len(aat_gains)))

2453
4906
7359
9812
12265
14718
17171
19624
22077
24530
-2627.999999999977
Num buys: 97
Num sells: 150
Num trades: 247
Num wins: 57
Num losses: 190
Win streak: 4
Loss streak: 25
Average pips risked: 0.004520647773279345
Average aat gains: -10.63967611336023


In [ ]:
negative_aats = list(filter(lambda x: x < 0, aat_gains))
positive_aats = list(filter(lambda x: x > 0, aat_gains))
zero_aats = list(filter(lambda x: x == 0, aat_gains))

In [ ]:
len(negative_aats)

In [ ]:
len(positive_aats)

In [ ]:
len(zero_aats)

In [ ]:
sum(negative_aats)

In [ ]:
sum(positive_aats)

In [ ]:
sum(positive_aats) + sum(negative_aats)